* Decorator is a function that takes another function as input, extends its behavior, and returnns a new function as output.
* Python functions are first-class objects, which means that they can be passed around and used as arguments, just like any other object (string, int, float, list, and so on).
* It should be noted that a decorator can also be used to decorate a class. A common use case is to use a class decorator as a registry for the classes created.

In [2]:
def func1():
    print("Inside func1")

In [4]:
func1()

Inside func1


In [6]:
from typing import Callable

def decorator(func: Callable):
    print("Inside the decorator")
    def wrapper():
        print("\tInside the wrapper")
        print("\tDo something before the `func` is executed")
        value = func()  # execute the `func`
        print("\tAlso inside the wrapper")
        print("\tDo something after the `func` is executed")
        
        # we need to return the value explicitly
        return value
    
    # Return the wrapper which is the wrapped/decorated function
    return wrapper

In [7]:
func1_decorated = decorator(func1)

Inside the decorator


In [8]:
func1_decorated()

	Inside the wrapper
	Do something before the `func` is executed
Inside func1
	Also inside the wrapper
	Do something after the `func` is executed


<hr>

* Let's introduce the magical `@` symbol, which is just the **syntactic sugar** that make the above function operation much easier.

In [9]:
@decorator  # call the decorator with the function as the argument
def func2():
    print("Inside func2")

Inside the decorator


In [10]:
func2()

	Inside the wrapper
	Do something before the `func` is executed
Inside func2
	Also inside the wrapper
	Do something after the `func` is executed


<hr>

* Now let's move on **decorator with arguments**. The following example is a decorator that takes an argument.

In [11]:
def decorator2(func: Callable):
    print("Inside the decorator2")
    
    # The parameters are passed to wrapper
    def wrapper(*args, **kwargs):
        print("\tInside the wrapper")
        print("\tDo something before the `func` is executed")
        
        value = func(*args, **kwargs)  # execute the `func`
        
        print("\tAlso inside the wrapper")
        print("\tDo something after the `func` is executed")
        
        # we need to return the value explicitly
        return value
    
    # Return the wrapper which is the wrapped/decorated function
    return wrapper

In [12]:
@decorator2
def multi_10(number: int, times: int = 1):
    print(f"Inside multi_10: {number} * {times} = {number * times} time(s)")

Inside the decorator2


In [13]:
multi_10(10, 2)

	Inside the wrapper
	Do something before the `func` is executed
Inside multi_10: 10 * 2 = 20 time(s)
	Also inside the wrapper
	Do something after the `func` is executed


* NOTE: There is a bit confused here.

In [14]:
print("multi_10.__name__:", multi_10.__name__)

multi_10.__name__: wrapper


> * The `multi_10.__name__` should be `multi_10` instead of `wrapper` because the `functools.wraps` decorator is used to copy the name, module, and docstring of the input function to the output function.

<hr>

* To resolve the above confusion, we can use the `functools.wraps` decorator to copy the name, module, and docstring of the input function to the output function.

In [15]:
from functools import wraps

def decorator3(func: Callable):
    print("Inside the decorator3")
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("\tInside the wrapper")
        print("\tDo something before the `func` is executed")
        
        value = func(*args, **kwargs)  # execute the `func`
        
        print("\tAlso inside the wrapper")
        print("\tDo something after the `func` is executed")
        
        # we need to return the value explicitly
        return value
    
    # Return the wrapper which is the wrapped/decorated function
    return wrapper

In [16]:
@decorator3
def multi_10_v2(number: int, times: int = 1):
    print(f"Inside multi_10_v2: {number} * {times} = {number * times} time(s)")

Inside the decorator3


In [17]:
multi_10_v2(10, 2)

	Inside the wrapper
	Do something before the `func` is executed
Inside multi_10_v2: 10 * 2 = 20 time(s)
	Also inside the wrapper
	Do something after the `func` is executed


In [18]:
print("multi_10_v2.__name__:", multi_10_v2.__name__)

multi_10_v2.__name__: multi_10_v2


<hr>

* **Decorator factory**: A decorator factory is a function that returns a decorator, which is a function that takes another function as input, extends its behavior, and returns a new function as output.

In [19]:
def decorator_factory(add_to_log = False):
    print("Inside the decorator_factory")
    
    # The function to decorate is passed to the decorator function
    def decorator(func: Callable):
        print("Inside the decorator")
        
        # The wraps decorator is used preserve the properties of the original function.
        # Note the parameters are passed to wrapper.
        @wraps(func)
        def wrapper(*args, **kwargs):
            print("\tInside the wrapper")
            print("\tDo something before the `func` is executed")
            
            value = func(*args, **kwargs)  # execute the `func`
            
            # The parameters passed to factory can be used in nested functions
            if add_to_log:
                print("\tInside the wrapper: add to log")
                print("\tDo something with the log, the function is logged after execution")
                
            # We need to return the value explicitly
            return value
        
        # Return the wrapper which is the wrapped/decorated function
        return wrapper
    
    # Return the decorator which is the decorator function
    return decorator

In [20]:
@decorator_factory(add_to_log=True)
def multi_10_v3(number: int, times: int = 1):
    print(f"Inside multi_10_v3: {number} * {times} = {number * times} time(s)")

Inside the decorator_factory
Inside the decorator


In [21]:
multi_10_v3(10, 2)

	Inside the wrapper
	Do something before the `func` is executed
Inside multi_10_v3: 10 * 2 = 20 time(s)
	Inside the wrapper: add to log
	Do something with the log, the function is logged after execution


In [22]:
@decorator_factory()
def multi_10_v4(number: int, times: int = 1):
    print(f"Inside multi_10_v4: {number} * {times} = {number * times} time(s)")

Inside the decorator_factory
Inside the decorator


<hr>

* Now we understand how decorator work, let's create a decorator which can be used to track the execution time of a function.

In [24]:
from functools import wraps
from datetime import datetime, timedelta

def debug_timer(exec_time: bool=True):
    def decorator(func: Callable):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start_time = datetime.now()
            value = func(*args, **kwargs)
            end_time = datetime.now()
            if exec_time:
                print(f"Execution time: {end_time - start_time}")
            return value
        return wrapper
    return decorator

In [25]:
@debug_timer()
def multi_10_v5(number: int, times: int = 1):
    print(f"Inside multi_10_v5: {number} * {times} = {number * times} time(s)")

In [26]:
multi_10_v5(10, 5)

Inside multi_10_v5: 10 * 5 = 50 time(s)
Execution time: 0:00:00.009047
